<a href="https://colab.research.google.com/github/Lafaralia/Binance_Futures_python/blob/master/Bot_offline_yfinance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import config
from binance.client import Client
from binance.enums import *
import time
import math
import datetime
import numpy as np
API_KEY = 'SB7ZuqA8Y8RYLq7mtkQsgWXlbzkkNASgCOsCTJdCPrXk5FpLjxzYVax20mvcxgcp'
API_SECRET = 'fsn7bQTKRyjPAqcQLXEkE6GkPQ6CFbfMGKe8sSpzOFTdEN6TpLJcs5yQOMGqS7iR'
client = Client(API_KEY, API_SECRET, tld='com')

symbolTicker = 'SOLBUSD'
stablecoin = symbolTicker[-4:]
quantity = 1.14
decimales = 2
margen = 2
margenC= 1+margen/100
margenV= 1-margen/100
prev_symbolPrice = 0
perdidas = 0
ganancias = 0
profittotal = 0
totaldecompras = 0
totaldeventas = 0
print(symbolTicker, '\nCantidad:',quantity )
#balance = client.get_asset_balance(asset=stablecoin)
#balanceant = float(balance['free'])
#print('Balance ', stablecoin, ':', round(balanceant,2))

while 1:
    # muestra el Balance actual
    #balance = client.get_asset_balance(asset=stablecoin)
    #balanceparcial = float(balance['free'])
    #balanceant = free
    #print('Balance BUSD: ', balanceant)
    
    ################# inicia compra, ojalá con el precio bajando
    list_of_tickers = client.get_all_tickers()
    for tick_2 in list_of_tickers:
        if tick_2["symbol"] == symbolTicker:
            prev_symbolPrice = float(tick_2["price"])


    #print('Inicia compra de ', symbolTicker)
    buyOrder = client.create_order(
            symbol = symbolTicker,
            side = 'BUY',
            type = 'STOP_LOSS_LIMIT',
            quantity = quantity,
            price = round(prev_symbolPrice*margenC,decimales),
            stopPrice = round(prev_symbolPrice*margenC*0.999,decimales),
            timeInForce = 'GTC')
    precioCompra=round(prev_symbolPrice*margenC,decimales)
    #montodeoperacion=precioCompra*quantity
    while 2:
        #balance = client.get_asset_balance(asset='BUSD')
        #free = float(balance['free'])
        #balanceant = free
        time.sleep(5)
        list_of_tickers = client.get_all_tickers()
        for tick_2 in list_of_tickers:
            if tick_2["symbol"] == symbolTicker:
                current_symbolPrice = float(tick_2["price"])
        
        #print("    Prev Price Buying= " + str(prev_symbolPrice))
        #print(" Current Price Buying= " + str(current_symbolPrice))
        #### Orden para salir del ciclo, si no hay ordenes abiertas       
        orders = client.get_open_orders(symbol=symbolTicker)
        if orders==[]:
                #totaldecompras += 1
                #montodeoperacion=precioCompra*quantity
                #print ('Precio de compra=', precioCompra)
                #if totaldecompras == 1: montodeapertura = montodeoperacion
                break
        
        if ( prev_symbolPrice > current_symbolPrice): #si va subiendo
            #cancela la OT activa
            result = client.cancel_order(
                symbol = symbolTicker,
                orderId = buyOrder.get('orderId')
            )
            #crea otra OT mas abajo
            buyOrder = client.create_order(
                    symbol = symbolTicker,
                    side = 'BUY',
                    type = 'STOP_LOSS_LIMIT',
                    quantity = quantity,
                    price = round(current_symbolPrice*margenC,decimales),
                    stopPrice = round(current_symbolPrice*margenC*0.999,decimales),
                    timeInForce = 'GTC')
            precioCompra=round(current_symbolPrice*margenC,decimales)
            prev_symbolPrice = current_symbolPrice
    if ( round(prev_symbolPrice,3) > round(current_symbolPrice,3 )):
        time.sleep(0)
    else:
        time.sleep(15)
             
             
             
    ###################### INICIA LA VENTA
    list_of_tickers = client.get_all_tickers()
    for tick_2 in list_of_tickers:
            if tick_2["symbol"] == symbolTicker:
                prev_symbolPrice = float(tick_2["price"])
    #print('Inicia venta de ', symbolTicker)
    sellOrder = client.create_order(
                symbol = symbolTicker,
                side = 'SELL',
                type = 'STOP_LOSS_LIMIT',
                quantity = quantity,
                price = round(prev_symbolPrice*margenV,decimales),
                stopPrice = round(prev_symbolPrice*margenV*1.001,decimales),
                timeInForce = 'GTC')
    
    #orderOCO = client.order_oco_sell(
                #symbol = symbolTicker,
                #quantity = quantity,
                #price = round(float(symbolPrice)*1.03,8),
                #stopPrice = round(float(symbolPrice)*0.991,8),
                #stopLimitPrice = round(float(symbolPrice)*0.99,8),
                #stopLimitTimeInForce = 'GTC'
    
    
    precioVenta = round(prev_symbolPrice,decimales)
    
    while 3:
        time.sleep(3)

        list_of_tickers = client.get_all_tickers()
        for tick_2 in list_of_tickers:
                if tick_2["symbol"] == symbolTicker:
                    current_symbolPrice = float(tick_2["price"])

        #print("    Prev Price Selling= " + str(prev_symbolPrice))
        #print(" Current Price Selling= " + str(current_symbolPrice))
        orders = client.get_open_orders(symbol=symbolTicker)
        # Muestra el balance y el Profit despues de la venta y sale del ciclo si ya no hay OT montada
        if orders==[]:
                print('Precio de compra: ', precioCompra)
                #balance = client.get_asset_balance(asset=stablecoin)
                #free = float(balance['free'])
                print('Precio de venta: ', precioVenta)
                montodeoperacion = quantity*precioCompra
                profit= (precioVenta*quantity)-montodeoperacion
                
                profittotal += profit
                porcprofit = round(100*profittotal/(montodeoperacion), 2)
                if  profit <= 0:
                        perdidas += 1
                else:
                        ganancias += 1
                print('Perdidas:', perdidas, ' Ganadas: ', ganancias)
                print('Monto de Operacion: ', round(montodeoperacion,2), '\nProfit: ', round(profit,4), porcprofit, '%')
                print('Perdidas:', perdidas, ' Ganadas: ', ganancias)
                print('Profit Total: ', round(profittotal,2), stablecoin )
                break
        if ( prev_symbolPrice < current_symbolPrice): # Subiendo
                result = client.cancel_order(
                        symbol = symbolTicker,
                        orderId = sellOrder.get('orderId')
                    )
                sellOrder = client.create_order(
                            symbol = symbolTicker,
                            side = 'SELL',
                            type = 'STOP_LOSS_LIMIT',
                            quantity = quantity,
                            price = round(prev_symbolPrice*margenV,decimales),
                            stopPrice = round(prev_symbolPrice*margenV*1.001,decimales),
                            timeInForce = 'GTC')
                precioVenta = round(prev_symbolPrice*margenV,decimales)
                prev_symbolPrice = current_symbolPrice
    #time.sleep(3)
    if ( round(prev_symbolPrice,3) < round(current_symbolPrice,3 )): #Subiendo
        time.sleep(0)
    else:
        time.sleep(15)